In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense
import json
import pickle
import os
from pathlib import Path

2025-04-16 23:34:29.282405: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 23:34:29.314252: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 23:34:29.314282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 23:34:29.315217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 23:34:29.320578: I tensorflow/core/platform/cpu_feature_guar

In [3]:
def build_path(*path_parts):
    base_dir = Path.cwd().parent
    return os.path.join(base_dir, *path_parts)

In [4]:
X_test = np.load(build_path('Data', 'X_test.npy'))
y_test = np.load(build_path('Data', 'y_test.npy'))
X_train = np.load(build_path('Data', 'X_train.npy'))
y_train = np.load(build_path('Data', 'y_train.npy'))

# Print the shape of the data
print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')


Training data shape: (40000, 400), Training labels shape: (40000,)
Testing data shape: (10000, 400), Testing labels shape: (10000,)


In [5]:
## Train Simple RNN
max_features = 10000 ## vocabulary size
max_len = 400 ## max length of the sequence
model=Sequential()
model.add(Embedding(max_features,128,input_length=max_len)) ## Embedding Layers
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation="sigmoid")) ## Output Layer

2025-04-16 23:34:39.148094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-16 23:34:39.186535: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
## Create an instance of EarlyStoppping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystopping

In [9]:
## Train the model with early sstopping
history=model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping],
)

Epoch 1/10


2025-04-16 23:34:42.945427: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential/simple_rnn/while/body/_1/sequential/simple_rnn/while/simple_rnn_cell/Relu}}
	.  Registered:  <no registered kernels>

2025-04-16 23:34:42.966920: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:135] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential/simple_rnn/while/body/_1/sequential/simple_rnn/while/simple_rnn_cell/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2025-04-16 23:34:42.984432: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: NOT_FOUND: Op type not registered '' in binary running on archlinux. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops fro

1000/1000 [==============================] - ETA: 0s - loss: 68124.0469 - accuracy: 0.6400

2025-04-16 23:35:49.507189: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential/simple_rnn/while/body/_1/sequential/simple_rnn/while/simple_rnn_cell/Relu}}
	.  Registered:  <no registered kernels>

2025-04-16 23:35:49.514350: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:135] tfg_optimizer{any(tfg-consolidate-attrs,tfg-toposort,tfg-shape-inference{graph-version=0},tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential/simple_rnn/while/body/_1/sequential/simple_rnn/while/simple_rnn_cell/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2025-04-16 23:35:49.520259: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: NOT_FOUND: Op type not registered '' in binary running on archlinux. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops fro

1000/1000 [==============================] - 71s 70ms/step - loss: 68124.0469 - accuracy: 0.6400 - val_loss: 0.4776 - val_accuracy: 0.7940
Epoch 2/10
1000/1000 [==============================] - 63s 63ms/step - loss: 0.4098 - accuracy: 0.8196 - val_loss: 0.3818 - val_accuracy: 0.8353
Epoch 3/10
1000/1000 [==============================] - 65s 65ms/step - loss: 0.3061 - accuracy: 0.8740 - val_loss: 0.3779 - val_accuracy: 0.8319
Epoch 4/10
1000/1000 [==============================] - 63s 63ms/step - loss: 0.2698 - accuracy: 0.8900 - val_loss: 0.4093 - val_accuracy: 0.8207
Epoch 5/10
1000/1000 [==============================] - 64s 64ms/step - loss: 0.2476 - accuracy: 0.9041 - val_loss: 0.4330 - val_accuracy: 0.8134
Epoch 6/10
1000/1000 [==============================] - 64s 64ms/step - loss: 0.2359 - accuracy: 0.9094 - val_loss: 0.4474 - val_accuracy: 0.8410
Epoch 7/10
1000/1000 [==============================] - 64s 64ms/step - loss: 0.1890 - accuracy: 0.9268 - val_loss: 0.4967 - val_ac

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


313/313 [==============================] - 4s 12ms/step - loss: 0.3687 - accuracy: 0.8398
Test Accuracy: 0.8398


In [11]:
## Save model file
model.save(build_path('Model', 'Model_scratch.h5'))

/mnt/utsav-raj/Project/IMDB/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
